In [ ]:
# Code to test whether this ipynb was already ran before
a = 5 
a

This code is used to infer transit shape from input light curve.
It involves following steps:
1. Load ML model
2. Load input data - i.e. the light curves
3. Predict the shapes from the light curves

4. Load True Output (if any)

Load ML model

In [ ]:
# Import TF and check for GPU

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

# Import required libraries

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import save_model, load_model
import math
import sys
from numpy import array,append,arange,zeros,exp,sin,random,std
from scipy.interpolate import interp1d

In [ ]:
model_load = load_model("../ml_model/nov13_model1/nov13_model1_sample_interpolate_no_noise_unf_samplept_20to60.h5")


Load Input data to infer on

In [ ]:
# 1. Load Lightcurve Dataset
lc_dir = '/scratch/abraham/Documents/mega_git/mega/data/random/npy/lc/lc_1_14Feb2024_circle_38_38_px.npy'
test_lc = np.load(lc_dir)

# Normalize the lightcurves
## - Test Set
test_lc_scaled = np.zeros(test_lc.shape)
for i in np.arange(len(test_lc_scaled)):
    test_lc_scaled[i] = (test_lc[i] - np.amin(test_lc[i]))/(np.amax(test_lc[i]) - np.amin(test_lc[i]))

# Append with ones
## - Test Set
test_lc_scaled_append = np.ones((test_lc.shape[0],120))
for i in np.arange(len(test_lc_scaled)):
    test_lc_scaled_append[i,10:110] = test_lc_scaled[i]
print("Extended the light curves")

Predict Shape

In [ ]:
# Test on the ML model - Test Dataset - Original Simulated light curve 
# (i.e. without sampling and interpolation)
# Test dataset - Prediciton
test_predict_shape = model_load.predict(test_lc_scaled_append)
test_predict_shape_normalized = np.zeros(test_predict_shape.shape)
for i in np.arange(len(test_predict_shape)):
    test_predict_shape_normalized[i] = (test_predict_shape[i] - np.amin(test_predict_shape[i]))/(np.amax(test_predict_shape[i]) - np.amin(test_predict_shape[i]))
print("Normalized the predicted shape")
test_predict_shape = test_predict_shape_normalized
del test_predict_shape_normalized

Plot (a single set) - Adjust index k to get the kth I/O
1. input light curve 
2. predicted output

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))  # 1 row, 2 columns
ph = np.linspace(-1,1,len(test_lc_scaled_append[0]))

# Plot on each subplot
k = int(0) # Index
axes[0].set_title('Light Curve')
axes[0].plot(ph, test_lc_scaled_append[k], color='blue')


axes[1].set_title('Predicted Shape')
axes[1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
img = axes[1].imshow(test_predict_shape[k],cmap='inferno')
plt.colorbar(img)

plt.show()


Load True Output if any

In [ ]:
shape_dir = '/scratch/abraham/Documents/mega_git/mega/data/test/npy/shape/shape_1.npy'
test_shape = np.load(shape_dir)

# Normalize the image, convert to opacity map
## Test Set
test_shape = test_shape/np.amax(test_shape)
test_shape_where_0 = np.where(test_shape == 0)
test_shape_where_1 = np.where(test_shape == 1)
test_shape[test_shape_where_0] = 1  # 1 represent the shape (1 opacity)
test_shape[test_shape_where_1] = 0  # 0 represent background (0 opacity)

Miscellance Operations
1. Plot input lightcurve
2. Show predicted output
3. Show true output

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns
ph = np.linspace(-1,1,len(test_lc_scaled_append[0]))

# Plot on each subplot
k = int(0) # Index
axes[0].set_title('Light Curve')
axes[0].plot(ph, test_lc_scaled_append[k], color='blue')


axes[1].set_title('Predicted Shape')
axes[1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
img = axes[1].imshow(test_predict_shape[k],cmap='inferno')
plt.colorbar(img)

axes[2].set_title('Actual Shape')
axes[2].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
img = axes[2].imshow(test_shape[k],cmap='inferno')
plt.colorbar(img)

plt.show()